### Imports

In [1]:
import cv2
import torch
import numpy as np
import pandas as pd
from PIL import Image, ImageEnhance, ImageFilter
from PIL.ImageOps import invert
from own_utils import remove_overlapping_junctions, non_max_suppression_fast

In [2]:
show_intermediate = False   # set to True to show intermediate results
output = []                 # create list to store all final outputs

### Image preprocessing

In [3]:
test_image = '../assets/example_image.jpg'

# convert to grayscale
img = Image.open(test_image).convert('L')
img = invert(img)
img = ImageEnhance.Contrast(img).enhance(2)
img = img.point(lambda p: p > 220 and 255)
img = img.filter(ImageFilter.SMOOTH)

# show image
if show_intermediate:
    img.show()

### Components Inference

In [4]:
# Get the model
model_path = '../models/components.pt' 
c_model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)
c_model.eval()

Using cache found in /home/tim/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-5-22 Python-3.10.12 torch-2.2.1+cu121 CPU

Fusing layers... 
Model summary: 212 layers, 20909508 parameters, 0 gradients, 48.0 GFLOPs
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 48, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(48, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(96, 48, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(96, 48, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(48, 48, kernel_size=(1, 1), stride=(1, 1))
  

In [5]:
# Inference
c_results = c_model(img)

# Create a new dataframe with the results
for result in c_results.xyxy:
    output.append([result[5], result[0], result[1], result[2], result[3], result[4]])

# Print and show results
if show_intermediate:
    print(c_results.pandas().xyxy)
    c_results.show()

### Junction inference

In [6]:
model_path = '../models/junctions.pt'
j_model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)
j_model.eval()

Using cache found in /home/tim/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-5-22 Python-3.10.12 torch-2.2.1+cu121 CPU

Fusing layers... 
Model summary: 212 layers, 20885262 parameters, 0 gradients, 48.0 GFLOPs
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 48, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(48, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(96, 48, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(96, 48, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(48, 48, kernel_size=(1, 1), stride=(1, 1))
  

In [7]:
# Perform inference on the image without components
j_results = j_model(img)

# Print and show results
if show_intermediate:
    print(j_results.pandas().xyxy)
    j_results.show()

### Post processing

In [8]:
# Remove overlapping junctions
c_coords, j_coords = remove_overlapping_junctions(j_results, c_results, overlap_threshold=0.1)

# Remember the old junctions
columns = ['xmin', 'ymin', 'xmax', 'ymax', 'confidence', 'class']
old_j_df = pd.DataFrame(j_coords, columns=columns)

# Perform non-maximum suppression on junctions
j_coords = non_max_suppression_fast(j_coords, iou_threshold=0.5)

# Create dataframes from the components and remaining junctions
c_df = pd.DataFrame(c_coords, columns=columns)
j_df = pd.DataFrame(j_coords, columns=columns)

if show_intermediate:
    print("Components:")
    print(c_df)

    print("Old junctions:")
    print(old_j_df)

    print("NMS Junctions:")
    print(j_df)

### Show final detections

In [9]:
# using c_df containing the components and j_df containing the junctions, draw all bounding boxes on the original image
img = cv2.imread(test_image)
c_labels = c_model.model.names
j_labels = j_model.model.names

# Draw components
for index, row in c_df.iterrows():
    xmin = int(row['xmin'])
    ymin = int(row['ymin'])
    xmax = int(row['xmax'])
    ymax = int(row['ymax'])
    cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)

    label = c_labels[int(row['class'])]
    cv2.putText(img, label, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)


# Draw junctions
for index, row in j_df.iterrows():
    xmin = int(row['xmin'])
    ymin = int(row['ymin'])
    xmax = int(row['xmax'])
    ymax = int(row['ymax'])
    cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0, 0, 255), 2)

    label = j_labels[int(row['class'])]
    cv2.putText(img, label, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)

# Save the image
cv2.imwrite('../assets/output.jpg', img)

{0: 'Ammeter', 1: 'ac_src', 2: 'battery', 3: 'cap', 4: 'curr_src', 5: 'dc_volt_src_1', 6: 'dc_volt_src_2', 7: 'dep_curr_src', 8: 'dep_volt', 9: 'diode', 10: 'gnd_1', 11: 'gnd_2', 12: 'inductor', 13: 'resistor', 14: 'voltmeter'}
{0: 'Ammeter', 1: 'ac_src', 2: 'battery', 3: 'cap', 4: 'curr_src', 5: 'dc_volt_src_1', 6: 'dc_volt_src_2', 7: 'dep_curr_src', 8: 'dep_volt', 9: 'diode', 10: 'gnd_1', 11: 'gnd_2', 12: 'inductor', 13: 'resistor', 14: 'voltmeter'}
{0: 'Ammeter', 1: 'ac_src', 2: 'battery', 3: 'cap', 4: 'curr_src', 5: 'dc_volt_src_1', 6: 'dc_volt_src_2', 7: 'dep_curr_src', 8: 'dep_volt', 9: 'diode', 10: 'gnd_1', 11: 'gnd_2', 12: 'inductor', 13: 'resistor', 14: 'voltmeter'}
{0: 'Ammeter', 1: 'ac_src', 2: 'battery', 3: 'cap', 4: 'curr_src', 5: 'dc_volt_src_1', 6: 'dc_volt_src_2', 7: 'dep_curr_src', 8: 'dep_volt', 9: 'diode', 10: 'gnd_1', 11: 'gnd_2', 12: 'inductor', 13: 'resistor', 14: 'voltmeter'}
{0: 'Ammeter', 1: 'ac_src', 2: 'battery', 3: 'cap', 4: 'curr_src', 5: 'dc_volt_src_1', 6

True

### Convert to generated image
Take the final output list and generate the digital circuit based on that

In [10]:
# TODO